In [1]:
library(Seurat)
library(ggplot2)
library(enrichR)
library(stringr)
library(RColorBrewer)
library(pheatmap)
library(dplyr)
library(scales)
library(dplyr)
library(ComplexHeatmap)
library(circlize)
library(tidyr)
library(tibble)

Warning message:
“package ‘Seurat’ was built under R version 4.3.3”
Loading required package: SeuratObject

Warning message:
“package ‘SeuratObject’ was built under R version 4.3.3”
Loading required package: sp

Warning message:
“package ‘sp’ was built under R version 4.3.3”

Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t


Warning message:
“package ‘ggplot2’ was built under R version 4.3.3”
Welcome to enrichR
Checking connection ... 

Enrichr ... 
Connection is Live!

FlyEnrichr ... 
Connection is Live!

WormEnrichr ... 
Connection is Live!

YeastEnrichr ... 
Connection is Live!

FishEnrichr ... 
Connection is Live!

OxEnrichr ... 
Connection is Live!

Warning message:
“package ‘stringr’ was built under R version 4.3.3”
Warning message:
“package ‘RColorBrewer’ was built under R version 4.3.3”
Warning message:
“package ‘dplyr’ was built under R version 4.3.3”

Attaching package: ‘dplyr’


The following objects are masked from

In [ ]:
ifnb = readRDS("hc_rbd_pd_joint_integrated.rds")

In [3]:
ifnb_testct <- ifnb

In [5]:
glimpse(ifnb_testct@meta.data$celltype)

 chr [1:231216] "excitatory neurons" "oligodendrocyte precursor cells" ...


# DEG analysis (PD vs HC) for each cell type

In [ ]:
set.seed(384)
Idents(ifnb_testct) <- "celltype"
deg_results_by_cluster <- list()


for (ct in unique(ifnb_testct$celltype)) {
  ct_subset <- subset(ifnb_testct, idents = ct)
  Idents(ct_subset) <- "disease"
  
  # Check if both "PD" and "HC" are present in the subset
  if (all(c("PD", "HC") %in% unique(Idents(ct_subset)))) {
    # Perform differential expression analysis between "PD" and "HC" within this cluster
    deg_results <- FindMarkers(ct_subset, ident.1 = "PD", ident.2 = "HC")
    deg_results_by_cluster[[ct]] <- deg_results
  } else {
    message(paste("Skipping ct", ct, "- does not contain both PD and HC cells"))
  }
}

In [ ]:
library(openxlsx)

# Create a new workbook
wb <- createWorkbook()

for (ct in unique(ifnb_testct$celltype)) {
    if (ct != "pericytes"){
        # Filter the data for each cluster
        cluster_data <- deg_results_by_cluster[[ct]] %>% filter(p_val_adj < 0.05) %>% arrange(desc(avg_log2FC))
        
        # Add a worksheet with the cluster name
        addWorksheet(wb, sheetName = ct)
        
        # Write the filtered data to the worksheet
        writeData(wb, sheet = ct, cluster_data, rowNames = TRUE)
    }
}

# Save the workbook to an Excel file
saveWorkbook(wb, file = "../../DEG_GO_Analysis/DEGs_PD_vs_HC/Significant_DEGs_PD_vs_HC_ct.xlsx", overwrite = TRUE)


In [ ]:
library(openxlsx)

xlsx_path <- "../../DEG_GO_Analysis/DEGs_PD_vs_HC/Significant_DEGs_PD_vs_HC_ct.xlsx"

# Get sheet names (these become the sublist names)
sheets <- getSheetNames(xlsx_path)

# Optionally drop any sheet you don't want
sheets <- setdiff(sheets, "pericytes")

# Read each sheet -> data.frame with gene rownames
deg_results_by_cluster <- setNames(
  lapply(sheets, function(sh) {
    df <- read.xlsx(xlsx_path, sheet = sh, rowNames = TRUE)

    # Make sure numeric columns are numeric (robust to minor name variants)
    nm <- tolower(names(df))
    num_cols <- names(df)[nm %in% c("p_val","avg_log2fc","avg_log2_fc","avg_logfc",
                                    "pct.1","pct1","pct.2","pct2","p_val_adj","padj","p_adj")]
    for (cc in num_cols) df[[cc]] <- as.numeric(df[[cc]])

    df
  }),
  sheets
)

# quick checks
names(deg_results_by_cluster)
# e.g. deg_results_by_cluster[["astrocytes"]][1:5, ]

[1] "excitatory neurons"              "oligodendrocyte precursor cells"
[3] "oligodendrocytes"                "inhibitory neurons"             
[5] "astrocytes"                      "microglia"                      
[7] "endothelia cells"

### Dot plot of top DEGs per cell type

In [5]:
top10_tables <- list()

for (cell_type in names(deg_results_by_cluster)) {
  cat("\nTop 10/20 genes for cell type:", cell_type, "\n")
  
  # Get the dataframe
  df <- deg_results_by_cluster[[cell_type]] %>% filter(p_val_adj < 0.05)
    
  # Add gene names as a column if they're rownames
  df$gene <- rownames(df)
    
  df <- df[!grepl("^(LINC)", df$gene), ]
  df <- df[!grepl("^(NPIPA8|TTTY)", df$gene), ]
  df <- df[!grepl("^(MT-)", df$gene), ]
  df <- df[!grepl("\\.\\d+$", df$gene), ]
    
  if (cell_type %in% c("microglia", "inhibitory neurons", "astrocytes")){
      df <- df %>% filter(pct.1 >= 0.25 | pct.2 >= 0.25)
  }
  
  # Select top 10 genes with highest avg_log2FC
  top_genes <- df[order(-df$avg_log2FC), ][1:10, ]

  if (cell_type %in% c("inhibitory neurons", "excitatory neurons")){
     top_genes <- df[order(-df$avg_log2FC), ][1:20, ]
  }

  # Save to list
  top10_tables[[cell_type]] <- top_genes
  
  # Optionally print the top genes
  print(top_genes)
}


Top 10/20 genes for cell type: excitatory neurons 
                 p_val avg_log2FC pct.1 pct.2     p_val_adj     gene
CRNDE     4.537810e-15   2.321718 0.014 0.006  1.660884e-10    CRNDE
HIST1H1D  3.169666e-24   2.240203 0.010 0.002  1.160129e-19 HIST1H1D
MET      8.520966e-124   1.739333 0.098 0.036 3.118759e-119      MET
PRKY     4.600518e-246   1.713994 0.152 0.048 1.683836e-241     PRKY
NLGN4Y    0.000000e+00   1.569057 0.615 0.207  0.000000e+00   NLGN4Y
HLA-DMA   1.118747e-57   1.551068 0.077 0.037  4.094724e-53  HLA-DMA
PTGDR     2.083627e-15   1.537189 0.025 0.013  7.626283e-11    PTGDR
RPS4Y1   4.502661e-241   1.507591 0.180 0.065 1.648019e-236   RPS4Y1
ZFY      1.781120e-273   1.478723 0.204 0.075 6.519076e-269      ZFY
ZFY-AS1   2.801526e-20   1.453695 0.017 0.007  1.025387e-15  ZFY-AS1
CARTPT    1.363782e-20   1.427322 0.028 0.014  4.991580e-16   CARTPT
EIF1AY    2.426816e-56   1.404657 0.045 0.016  8.882388e-52   EIF1AY
KDM5D     0.000000e+00   1.371192 0.340 0.125  0.00

In [ ]:
custom_palette <- c('#2166ac','#92c5de','#f7f7f7','#f4a582','#b2182b')

# Loop through each cell type in the DEG results
for (cell_type in names(deg_results_by_cluster)) {
  top_genes <- top10_tables[[cell_type]][1:10,"gene"]
  if (cell_type %in% c("inhibitory neurons", "excitatory neurons")){
     top_genes <- top10_tables[[cell_type]][1:20,"gene"]
  }
  
  # Subset Seurat object for this cell type
  sub_obj <- subset(ifnb_testct, subset = celltype == cell_type)
  
  # Keep only genes that exist in this object
  available_genes <- intersect(top_genes, rownames(sub_obj))
  
  # Skip if no genes available
  if (length(available_genes) == 0) next

  # Get plot data first
  plot_obj <- DotPlot(
    object   = sub_obj,
    features = available_genes,
    group.by = "disease"
  )
  plot_data <- plot_obj$data
  
  # Calculate dynamic size breaks for percent expressed
  min_pct <- min(plot_data$pct.exp)
  max_pct <- max(plot_data$pct.exp)
  breaks <- seq(min_pct, max_pct, length.out = 5)
  labels <- paste0(round(breaks), "%")
  
  # Now plot with dynamic size scale
  p <- plot_obj +
    scale_color_gradientn(colors = custom_palette) +
    scale_size_area(
      max_size = 15,
      breaks = breaks,
      labels = labels,
      name = "Percent Expressed"
    ) +
    ggtitle(paste("Cell Type:", cell_type, "- Top 10/20 DEGs for PD vs HC")) +
    RotatedAxis() +
    theme(axis.text.x = element_text(angle = 45, hjust = 1))
  
  # Save
  out_file <- paste0("../../man_figs_pdf/deg_dot_plot/PD_vs_HC_CellType_", cell_type, ".pdf")
  ggsave(filename = out_file, plot = p, device = "pdf", width = 10, height = 5)
}

Warning message:
“Scaling data with a low number of groups may produce misleading results”
Scale for colour is already present.
Adding another scale for colour, which will replace the existing scale.
Scale for size is already present.
Adding another scale for size, which will replace the existing scale.
Warning message:
“Scaling data with a low number of groups may produce misleading results”
Scale for colour is already present.
Adding another scale for colour, which will replace the existing scale.
Scale for size is already present.
Adding another scale for size, which will replace the existing scale.
Warning message:
“Scaling data with a low number of groups may produce misleading results”
Scale for colour is already present.
Adding another scale for colour, which will replace the existing scale.
Scale for size is already present.
Adding another scale for size, which will replace the existing scale.
Warning message:
“Scaling data with a low number of groups may produce misleading res

### DEG counts per cell type - Heatmap

In [ ]:
# --- INPUTS ---------------------------------------------------------------
xlsx_path <- "../../DEG_GO_Analysis/DEGs_PD_vs_HC/Significant_DEGs_PD_vs_HC_ct.xlsx"
out_pdf   <- "../../man_figs_pdf/allct_top_deg_heatmap_PD_vs_HC.pdf"

row_order_words <- c(
  "astrocytes",
  "endothelia cells",
  "excitatory neurons",
  "inhibitory neurons",
  "microglia",
  "oligodendrocytes",
  "oligodendrocyte precursor cells"
)

abbr_map <- c(
  "excitatory neurons"               = "Exc",
  "inhibitory neurons"               = "Inh",
  "oligodendrocytes"                 = "Oli",
  "oligodendrocyte precursor cells"  = "OPC",
  "endothelia cells"                 = "Endo",
  "endothelial cells"                = "Endo",  # alias just in case
  "astrocytes"                       = "Ast",
  "microglia"                        = "Mic"
)

top_n_per_group <- 10  # << top 10 per cell type

# --- 1) COLLECT THE UNION OF TOP GENES (ORDERED BY row_order_words) ---------
stopifnot(exists("top10_tables"), length(top10_tables) > 0)
tt_norm <- setNames(top10_tables, tolower(names(top10_tables)))

grouped_genes <- list()
for (ct in row_order_words) {
  key <- tolower(ct)
  if (!key %in% names(tt_norm)) next
  df <- tt_norm[[key]]
  genes_ct <- if ("gene" %in% names(df)) df$gene else rownames(df)
  genes_ct <- genes_ct[!is.na(genes_ct)]
  genes_ct <- unique(genes_ct)
  grouped_genes[[ct]] <- head(genes_ct, top_n_per_group)  # enforce top 10 per group
}

gene_order <- unlist(grouped_genes, use.names = FALSE)
stopifnot(length(gene_order) > 0)

# vertical separators between gene groups
group_sizes <- vapply(grouped_genes, length, integer(1))
vlines_x <- cumsum(group_sizes)
if (length(vlines_x) > 0) vlines_x <- vlines_x[-length(vlines_x)] + 0.5

# unique internal keys for columns; pretty labels can repeat
gene_label     <- gene_order
gene_key_order <- make.unique(gene_order, sep = "_dup")
names(gene_label) <- gene_key_order  # key -> label

# --- 2) READ LOG2FC FROM WORKBOOK ------------------------------------------
sheet_names <- getSheetNames(xlsx_path)

read_fc_vec <- function(sheet_nm) {
  df <- read.xlsx(xlsx_path, sheet = sheet_nm, rowNames = TRUE)
  if (nrow(df) == 0) return(setNames(numeric(0), character(0)))
  nm <- tolower(names(df))
  fc_col <- names(df)[match(TRUE, nm %in% c("avg_log2fc","avg_log2_fc","avg_logfc","log2fc"))]
  if (is.na(fc_col)) stop("avg_log2FC column not found in sheet: ", sheet_nm)
  g <- if ("gene" %in% names(df)) df$gene else rownames(df)
  setNames(as.numeric(df[[fc_col]]), g)
}

sheet_for <- function(ct_word) {
  idx <- which(tolower(sheet_names) == tolower(ct_word))
  if (length(idx) == 0 && tolower(ct_word) == "endothelia cells") {
    idx <- which(tolower(sheet_names) == "endothelial cells")
  }
  if (length(idx) == 0) return(NA_character_)
  sheet_names[idx[1]]
}

row_sheets <- vapply(row_order_words, sheet_for, character(1))
missing_rows <- is.na(row_sheets)
if (any(missing_rows)) {
  message("Warning: missing sheets for rows: ",
          paste(row_order_words[missing_rows], collapse = ", "))
  row_order_words <- row_order_words[!missing_rows]
  row_sheets      <- row_sheets[!missing_rows]
}

# build matrix (rows = cell types; cols = gene_key_order)
mat <- matrix(0, nrow = length(row_order_words), ncol = length(gene_key_order))
rownames(mat) <- row_order_words
colnames(mat) <- gene_key_order

# fill by aligning workbook vectors to gene_label (handles duplicates correctly)
for (i in seq_along(row_sheets)) {
  vec <- read_fc_vec(row_sheets[i])
  vec <- vec[!is.na(names(vec))]
  aligned <- vec[unname(gene_label)]      # repeats values for duplicated labels
  aligned[is.na(aligned)] <- 0
  mat[i, ] <- aligned
}

# y-axis abbreviations
row_labels <- unname(abbr_map[tolower(rownames(mat))])
row_labels[is.na(row_labels)] <- rownames(mat)[is.na(row_labels)]

# --- 3) LONG DATA + COLOR LIMITS --------------------------------------------
df_long <- as.data.frame(mat)
df_long$celltype <- factor(row_labels, levels = row_labels)

df_long <- tidyr::pivot_longer(
  df_long,
  cols = all_of(colnames(mat)),
  names_to = "gene_key",
  values_to = "log2fc"
)

all_vals <- df_long$log2fc[is.finite(df_long$log2fc)]
lo <- stats::quantile(all_vals, 0.02, na.rm = TRUE)
hi <- stats::quantile(all_vals, 0.98, na.rm = TRUE)
lim_low  <- min(lo, -0.5)
lim_high <- max(hi,  0.5)

lab_map <- setNames(unname(gene_label), names(gene_label))  # key -> label

# --- COLOR PALETTES ----------------------------------------------------------
pal_down <- c('#f7fbff','#deebf7','#c6dbef','#9ecae1','#6baed6','#4292c6','#2171b5','#08519c','#08306b')
pal_up   <- c('#fff5f0','#fee0d2','#fcbba1','#fc9272','#fb6a4a','#ef3b2c','#cb181d','#a50f15','#67000d')
pal_all  <- c(rev(pal_down), "#ffffff", pal_up)

p0  <- (0 - lim_low) / (lim_high - lim_low)
eps <- 1e-6
vals <- c(
  seq(0, max(0, p0 - eps), length.out = length(pal_down)),
  p0,
  seq(min(1, p0 + eps), 1, length.out = length(pal_up))
)

# --- 4) PLOT -----------------------------------------------------------------
p <- ggplot(df_long, aes(x = gene_key, y = celltype, fill = log2fc)) +
  geom_tile(color = "black", linewidth = 0.25) +
  scale_fill_gradientn(colors = pal_all, values = vals,
                       limits = c(lim_low, lim_high),
                       name = expression(log[2]*"FC"),
                       na.value = "#ffffff") +
  scale_x_discrete(position = "top",
                   limits = gene_key_order,
                   labels = lab_map[gene_key_order]) +
  scale_y_discrete(position = "left", limits = rev(row_labels)) +
  labs(x = NULL, y = NULL) +
  theme_minimal(base_size = 12) +
  theme(
    panel.grid = element_blank(),
    axis.text.y = element_text(face = "bold"),
    axis.text.x = element_text(angle = 90, hjust = 0, vjust = 0.5, face = "italic"),
    legend.title = element_text(face = "bold")
  )

if (length(vlines_x) > 0) {
  p <- p + geom_vline(xintercept = vlines_x, color = "black", linewidth = 0.9)
}

w <- max(6, 0.25 * length(gene_key_order) + 2)
h <- 2.2 + 0.45 * length(row_order_words)

ggsave(out_pdf, plot = p, width = w, height = h, units = "in", device = "pdf")
message("Saved: ", out_pdf)

Saved: ../man_figs_pdf/allct_top_deg_heatmap_PD_vs_HC.pdf



## GO Enrichment analysis on DEGs for each cell type

In [6]:
go_analysis <- function(deg_results_by_cluster, log2FC_threshold = 0.0){
    set.seed(384)
    # Define the database(s) to use for EnrichR, focusing on GO Biological Process
    dbs <- c("GO_Biological_Process_2021")
    
    # Initialize a list to store enrichment results
    enrichment_results <- list()
    marker_counts <- list() 
    
    # Loop over each cluster in deg_results_by_cluster to perform enrichment analysis
    for (cluster_name in names(deg_results_by_cluster)) {
      # Retrieve DEGs for the current cluster from deg_results_by_cluster
      cluster_data <- deg_results_by_cluster[[cluster_name]]
      
      # Select significant DEGs (e.g., adjusted p-value < 0.05 and log fold change threshold)
      significant_genes <- rownames(cluster_data[cluster_data$p_val_adj < 0.05 & cluster_data$avg_log2FC > log2FC_threshold, ])
      marker_counts[[cluster_name]] <- length(significant_genes)
        
      # Perform enrichment analysis if there are significant genes
      if (length(significant_genes) > 0) {
        enrich_result <- enrichr(significant_genes, dbs)
        go_results <- enrich_result[["GO_Biological_Process_2021"]]
        
        # Sort results by adjusted p-value and select the top 10 processes
        go_results <- go_results[order(go_results$Adjusted.P.value), ]
        top_processes <- head(go_results, 15)
    
        if (nrow(top_processes) != 0) {
            # Initialize list to store the average fold change for each GO term
            avg_fc_for_terms <- numeric(nrow(top_processes))
            
            # Calculate the average fold change for each GO term in top_processes
            for (j in 1:nrow(top_processes)) {
              if (!is.na(top_processes$Genes[j]) && top_processes$Genes[j] != "") {
                genes_in_term <- strsplit(as.character(top_processes$Genes[j]), ";")[[1]]  # Split the genes string by semicolon
                
                # Only keep genes that are present in cluster_data
                genes_in_term <- genes_in_term[genes_in_term %in% rownames(cluster_data)]
                  
                # Retrieve fold changes for these genes
                if (length(genes_in_term) > 0) {
                  fc_values <- cluster_data[genes_in_term, "avg_log2FC"]
                  avg_fc_for_terms[j] <- mean(fc_values, na.rm = TRUE)  # Calculate the average, ignoring NA values
                } else {
                  avg_fc_for_terms[j] <- NA  # If no genes are found, set to NA
                }
              } else {
                avg_fc_for_terms[j] <- NA  # If 'Genes' is NA, set to NA
              }
            }
        
            # Add the average fold changes as a new column in top_processes
            top_processes$Avg_Log2FC <- avg_fc_for_terms
        }
        # Store the top GO processes for this cluster
        enrichment_results[[cluster_name]] <- top_processes
      }
    }
    return(list(marker_counts = marker_counts, enrichment_results = enrichment_results))
}

In [7]:
set.seed(384)
deg_results_by_cluster_filt <- lapply(deg_results_by_cluster, function(df) {
  genes <- rownames(df)
  keep  <- (!grepl("^(RPL|RPS)", genes)) & (!grepl("^MT-", genes)) & (!grepl("\\.", genes))
  df[keep, , drop = FALSE]
})
output <- go_analysis(deg_results_by_cluster_filt, log2FC_threshold = 0.25)
list2env(output, envir = .GlobalEnv)

Uploading data to Enrichr... Done.
  Querying GO_Biological_Process_2021... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying GO_Biological_Process_2021... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying GO_Biological_Process_2021... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying GO_Biological_Process_2021... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying GO_Biological_Process_2021... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying GO_Biological_Process_2021... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying GO_Biological_Process_2021... Done.
Parsing results... Done.


<environment: R_GlobalEnv>

In [8]:
marker_counts

$`excitatory neurons`
[1] 281

$`oligodendrocyte precursor cells`
[1] 159

$oligodendrocytes
[1] 889

$`inhibitory neurons`
[1] 330

$astrocytes
[1] 1557

$microglia
[1] 400

$`endothelia cells`
[1] 29

In [ ]:
set.seed(384) 
# Prepare matrices for plotting
cluster_names <- c("astrocytes", "endothelia cells", "excitatory neurons", "inhibitory neurons", "microglia", "oligodendrocytes", "oligodendrocyte precursor cells")
# Collect unique biological processes across all clusters
all_processes <- unique(unlist(lapply(enrichment_results, function(x) x$Term)))
enrichment_matrix <- matrix(
  NA, nrow = length(all_processes), ncol = length(cluster_names),
  dimnames = list(all_processes, cluster_names)
)
fc_matrix <- matrix(
  NA, nrow = length(all_processes), ncol = length(cluster_names),
  dimnames = list(all_processes, cluster_names)
)

# Fill the matrices with -log10(adjusted p-value) and Avg_Log2FC for each process and cluster
for (cluster_name in cluster_names) {
  if (!is.null(enrichment_results[[cluster_name]]) && nrow(enrichment_results[[cluster_name]]) > 0) {
    for (j in 1:nrow(enrichment_results[[cluster_name]])) {
      process <- enrichment_results[[cluster_name]]$Term[j]
      pval <- enrichment_results[[cluster_name]]$Adjusted.P.value[j]
      avg_log2fc <- enrichment_results[[cluster_name]]$Avg_Log2FC[j]
  
      # Fill p-value matrix with -log10(p-value)
      enrichment_matrix[process, cluster_name] <- -log10(pval)
      
      # Fill fold change matrix with Avg_Log2FC
      fc_matrix[process, cluster_name] <- avg_log2fc
      
    }
  }
}   

# Replace NA values with a low number (e.g., 0) for better plotting
enrichment_matrix[is.na(enrichment_matrix)] <- 0

output_pdf = "../../man_figs_pdf/allct_deg_go_PDvsHC_logp+.pdf"
# Plot the heatmap using pheatmap
pheatmap::pheatmap(enrichment_matrix,
                   color = c(
                     colorRampPalette(c("white", '#ef3b2c'))(10),  # More resolution for small values
                     colorRampPalette(c('#ef3b2c','#cb181d','#a50f15','#67000d'))(90) # High contrast for larger values
                   ),
                   na_col = "grey",           # Use grey for NA values (e.g., processes not enriched in clusters)
                   main = "Enriched BP of Microglial State Signatures (-log10 p-value)",
                   cluster_rows = FALSE,       # Cluster processes for better visualization
                   cluster_cols = FALSE,      # Do not cluster clusters (retain the original order)
                   angle_col = 45,            # Rotate column names for readability
                   fontsize_row = 6,          # Font size for process names
                   fontsize_col = 8,
                   filename = output_pdf)         # Font size for cluster labels
print("Logp Plot saved")

# Replace NA values with a low number (e.g., 0) for better plotting
fc_matrix[is.na(fc_matrix)] <- 0

# Define the custom color palettes
# negative_colors <- colorRampPalette(c('#f7fbff','#deebf7','#c6dbef','#9ecae1','#6baed6','#4292c6', '#2171b5', '#084594'))(50)
negative_colors <- colorRampPalette(c('#084594','#2171b5','#4292c6','#6baed6','#9ecae1','#c6dbef', '#deebf7', '#f7fbff'))(50)
positive_colors <- colorRampPalette(c('#fff5f0','#fee0d2','#fcbba1','#fc9272','#fb6a4a','#ef3b2c','#cb181d','#99000d'))(50)

# Combine palettes with white at 0
custom_colors <- c(negative_colors, "white", positive_colors)

# Define custom breaks to center the color scale at 0
max_abs_value <- max(abs(fc_matrix))
breaks <- seq(-max_abs_value, max_abs_value, length.out = length(custom_colors) + 1)

output_pdf = "../../man_figs_pdf/allct_deg_go_PDvsHC_log2fc+.pdf"
pheatmap::pheatmap(fc_matrix,
                   color = custom_colors,
                   breaks = breaks,              # Use custom breaks for diverging colors
                   na_col = "grey",              # Use grey for NA values
                   main = "Average Log2 Fold Change",
                   cluster_rows = FALSE,          # Cluster processes for better visualization
                   cluster_cols = FALSE,          # Cluster clusters for better visualization
                   angle_col = 45,               # Rotate column names for readability
                   fontsize_row = 6,             # Font size for process names
                   fontsize_col = 8,
                   filename = output_pdf)             # Font size for cluster labels
print("Log2FC File saved")                                                  

[1] "Logp Plot saved"
[1] "Log2FC File saved"


In [ ]:
library(openxlsx)
# Create a new workbook
wb <- createWorkbook()

for (cluster_name in cluster_names) {
    # Filter the data for each cluster
    cluster_data <- enrichment_results[[cluster_name]]
    
    # Add a worksheet with the cluster name
    addWorksheet(wb, sheetName = cluster_name)
    
    # Write the filtered data to the worksheet
    writeData(wb, sheet = cluster_name, cluster_data, rowNames = TRUE)
}

# Save the workbook to an Excel file
saveWorkbook(wb, file = "../../DEG_GO_Analysis/GO_PD_vs_HC/GO_Results_PD_vs_HC_allct+.xlsx", overwrite = TRUE)


In [ ]:
saveRDS(enrichment_results, "../../DEG_GO_Analysis/GO_PD_vs_HC/cached_enrichment_results_allct+.rds")

# DEG analysis (RBD vs HC) for each cell type

In [9]:
set.seed(384)
Idents(ifnb_testct) <- "celltype"
deg_results_by_cluster <- list()


for (ct in unique(ifnb_testct$celltype)) {
  ct_subset <- subset(ifnb_testct, idents = ct)
  Idents(ct_subset) <- "disease"
  
  # Check if both "PD" and "HC" are present in the subset
  if (all(c("RBD", "HC") %in% unique(Idents(ct_subset)))) {
    # Perform differential expression analysis between "PD" and "HC" within this cluster
    deg_results <- FindMarkers(ct_subset, ident.1 = "RBD", ident.2 = "HC")
    deg_results_by_cluster[[ct]] <- deg_results
  } else {
    message(paste("Skipping ct", ct, "- does not contain both RBD and HC cells"))
  }
}

In [ ]:
library(openxlsx)

# Create a new workbook
wb <- createWorkbook()

for (ct in unique(ifnb_testct$celltype)) {
    # Filter the data for each cluster
    cluster_data <- deg_results_by_cluster[[ct]] %>% filter(p_val_adj < 0.05) %>% arrange(desc(avg_log2FC))
    
    # Add a worksheet with the cluster name
    addWorksheet(wb, sheetName = ct)
    
    # Write the filtered data to the worksheet
    writeData(wb, sheet = ct, cluster_data, rowNames = TRUE)
}

# Save the workbook to an Excel file
saveWorkbook(wb, file = "../../DEG_GO_Analysis/DEGs_RBD_vs_HC/Significant_DEGs_RBD_vs_HC_ct.xlsx", overwrite = TRUE)


In [ ]:
library(openxlsx)

xlsx_path <- "../../DEG_GO_Analysis/DEGs_RBD_vs_HC/Significant_DEGs_RBD_vs_HC_ct.xlsx"

# Get sheet names (these become the sublist names)
sheets <- getSheetNames(xlsx_path)

# Optionally drop any sheet you don't want
sheets <- setdiff(sheets, "pericytes")

# Read each sheet -> data.frame with gene rownames
deg_results_by_cluster <- setNames(
  lapply(sheets, function(sh) {
    df <- read.xlsx(xlsx_path, sheet = sh, rowNames = TRUE)

    # Make sure numeric columns are numeric (robust to minor name variants)
    nm <- tolower(names(df))
    num_cols <- names(df)[nm %in% c("p_val","avg_log2fc","avg_log2_fc","avg_logfc",
                                    "pct.1","pct1","pct.2","pct2","p_val_adj","padj","p_adj")]
    for (cc in num_cols) df[[cc]] <- as.numeric(df[[cc]])

    df
  }),
  sheets
)

# quick checks
names(deg_results_by_cluster)
# e.g. deg_results_by_cluster[["astrocytes"]][1:5, ]

[1] "excitatory neurons"              "oligodendrocyte precursor cells"
[3] "oligodendrocytes"                "inhibitory neurons"             
[5] "astrocytes"                      "microglia"                      
[7] "endothelia cells"

### DEG counts per cell type - Heatmap

In [ ]:
suppressPackageStartupMessages({
  library(openxlsx)
  library(dplyr)
  library(tidyr)
  library(ggplot2)
  library(ggnewscale)  # install.packages("ggnewscale") if needed
})

# ---- paths ----
xlsx_path <- "../../DEG_GO_Analysis/DEGs_RBD_vs_HC/Significant_DEGs_RBD_vs_HC_ct.xlsx"
out_pdf   <- "../../man_figs_pdf/allct_DEG_counts_heatmap_RBD_vs_HC.pdf"

# ---- settings ----
fc_thresh <- 0.25
desired_order <- c(
  "astrocytes",
  "endothelia cells",
  "excitatory neurons",
  "inhibitory neurons",
  "microglia",
  "oligodendrocytes",
  "oligodendrocyte precursor cells"
)

# ---- read sheets and count ----
sheets <- setdiff(getSheetNames(xlsx_path), "pericytes")

count_list <- lapply(sheets, function(ct) {
  df <- read.xlsx(xlsx_path, sheet = ct, rowNames = TRUE)
  if (nrow(df) == 0) return(tibble(celltype = ct, Up = 0L, Down = 0L))

  nm <- tolower(names(df))
  fc_col   <- names(df)[which(nm %in% c("avg_log2fc","avg_log2_fc","avg_logfc","log2fc"))[1]]
  padj_col <- names(df)[which(nm %in% c("p_val_adj","padj","p_adj","p.adjusted"))[1]]
  if (is.na(fc_col)) stop("Could not find log2FC column in sheet: ", ct)

  fc <- as.numeric(df[[fc_col]])
  keep <- is.finite(fc) & abs(fc) > fc_thresh
  if (!is.na(padj_col)) keep <- keep & is.finite(df[[padj_col]]) & (as.numeric(df[[padj_col]]) < 0.05)

  fc <- fc[keep]

  tibble(
    celltype = ct,
    Up   = sum(fc > 0, na.rm = TRUE),
    Down = sum(fc < 0, na.rm = TRUE)
  )
})

counts <- bind_rows(count_list)

# ---- order rows per requested biology order (case-insensitive) ----
counts <- counts %>%
  mutate(.order_idx = match(tolower(celltype), tolower(desired_order))) %>%
  arrange(.order_idx, .by_group = FALSE) %>%
  select(-.order_idx)

# ---- long format ----
heat_df <- counts %>%
  mutate(celltype = factor(celltype, levels = rev(celltype))) %>%
  pivot_longer(c(Down, Up), names_to = "Direction", values_to = "Count")

# Palettes (independent scales for each column)
pal_down <- c('#f7fbff','#deebf7','#c6dbef','#9ecae1','#6baed6','#4292c6','#2171b5','#08519c','#08306b')
pal_up   <- c('#fff5f0','#fee0d2','#fcbba1','#fc9272','#fb6a4a','#ef3b2c','#cb181d','#a50f15','#67000d')

max_down <- max(heat_df$Count[heat_df$Direction == "Down"], na.rm = TRUE)
max_up   <- max(heat_df$Count[heat_df$Direction == "Up"],   na.rm = TRUE)

# ---- plot ----
p <- ggplot() +
  geom_tile(
    data = dplyr::filter(heat_df, Direction == "Down"),
    aes(x = Direction, y = celltype, fill = Count),
    color = "black", linewidth = 0.5
  ) +
  scale_fill_gradientn(colors = pal_down, limits = c(0, max_down)) +
  ggnewscale::new_scale_fill() +
  geom_tile(
    data = dplyr::filter(heat_df, Direction == "Up"),
    aes(x = Direction, y = celltype, fill = Count),
    color = "black", linewidth = 0.5
  ) +
  scale_fill_gradientn(colors = pal_up, limits = c(0, max_up)) +
  geom_text(data = heat_df, aes(x = Direction, y = celltype, label = Count), size = 3.6) +
  scale_x_discrete(position = "top") +
  labs(x = NULL, y = NULL) +
  theme_minimal(base_size = 12) +
  theme(
    panel.grid = element_blank(),
    legend.position = "none",
    axis.text.x = element_text(face = "bold", margin = margin(b = 4)),
    axis.text.y = element_text(face = "bold")
  )

ht <- max(3, 0.45 * length(levels(heat_df$celltype)) + 1)
ggsave(out_pdf, plot = p, width = 3.2, height = ht, units = "in", device = "pdf")

Warning message:
“package ‘ggnewscale’ was built under R version 4.3.3”


### Dot plot for top DEGs per cell type

In [26]:
top10_tables <- list()

for (cell_type in names(deg_results_by_cluster)) {
  cat("\nTop 10/20 genes for cell type:", cell_type, "\n")
  
  # Get the dataframe
  df <- deg_results_by_cluster[[cell_type]] %>% filter(p_val_adj < 0.05)
    
  # Add gene names as a column if they're rownames
  df$gene <- rownames(df)
    
  df <- df[!grepl("^(LINC)", df$gene), ]
  df <- df[!grepl("^(NPIPA8|TTTY)", df$gene), ]
  df <- df[!grepl("^(MT-)", df$gene), ]
  df <- df[!grepl("\\.\\d+$", df$gene), ]
    
  if (cell_type %in% c("microglia", "inhibitory neurons", "astrocytes")){
      df <- df %>% filter(pct.1 >= 0.25 | pct.2 >= 0.25)
  }
  
  # Select top 10 genes with highest avg_log2FC
  top_genes <- df[order(-df$avg_log2FC), ][1:10, ]

  if (cell_type %in% c("inhibitory neurons", "excitatory neurons")){
     top_genes <- df[order(-df$avg_log2FC), ][1:20, ]
  }

  # Save to list
  top10_tables[[cell_type]] <- top_genes
  
  # Optionally print the top genes
  print(top_genes)
}


Top 10/20 genes for cell type: excitatory neurons 
                 p_val avg_log2FC pct.1 pct.2     p_val_adj     gene
SERPINA3  2.499571e-37   3.189761 0.015 0.003  9.148681e-33 SERPINA3
CHI3L1    1.842128e-82   2.917046 0.044 0.011  6.742373e-78   CHI3L1
PRKY      0.000000e+00   2.667545 0.271 0.048  0.000000e+00     PRKY
CD44      5.211069e-48   2.573488 0.029 0.008  1.907303e-43     CD44
KIF25    1.085900e-111   2.381427 0.046 0.009 3.974503e-107    KIF25
TBL1Y     1.735052e-21   2.150881 0.011 0.003  6.350462e-17    TBL1Y
KDM5D     0.000000e+00   2.140323 0.542 0.125  0.000000e+00    KDM5D
USP9Y     0.000000e+00   2.121901 0.930 0.210  0.000000e+00    USP9Y
RPS4Y1    0.000000e+00   2.099902 0.270 0.065  0.000000e+00   RPS4Y1
NLGN4Y    0.000000e+00   2.093563 0.905 0.207  0.000000e+00   NLGN4Y
TMSB4Y    7.195475e-28   2.077732 0.013 0.003  2.633616e-23   TMSB4Y
CD24     2.429056e-110   2.056773 0.054 0.013 8.890588e-106     CD24
ZFY       0.000000e+00   2.002249 0.300 0.075  0.00

In [12]:
custom_palette <- c('#2166ac','#92c5de','#f7f7f7','#f4a582','#b2182b')

# Loop through each cell type in the DEG results
for (cell_type in names(deg_results_by_cluster)) {
  top_genes <- top10_tables[[cell_type]][1:10,"gene"]
  if (cell_type %in% c("inhibitory neurons", "excitatory neurons")){
     top_genes <- top10_tables[[cell_type]][1:20,"gene"]
  }
  
  # Subset Seurat object for this cell type
  sub_obj <- subset(ifnb_testct, subset = celltype == cell_type)
  
  # Keep only genes that exist in this object
  available_genes <- intersect(top_genes, rownames(sub_obj))
  
  # Skip if no genes available
  if (length(available_genes) == 0) next

  # Get plot data first
  plot_obj <- DotPlot(
    object   = sub_obj,
    features = available_genes,
    group.by = "disease"
  )
  plot_data <- plot_obj$data
  
  # Calculate dynamic size breaks for percent expressed
  min_pct <- min(plot_data$pct.exp)
  max_pct <- max(plot_data$pct.exp)
  breaks <- seq(min_pct, max_pct, length.out = 5)
  labels <- paste0(round(breaks), "%")
  
  # Now plot with dynamic size scale
  p <- plot_obj +
    scale_color_gradientn(colors = custom_palette) +
    scale_size_area(
      max_size = 15,
      breaks = breaks,
      labels = labels,
      name = "Percent Expressed"
    ) +
    ggtitle(paste("Cell Type:", cell_type, "- Top 10 DEGs for RBD vs HC")) +
    RotatedAxis() +
    theme(axis.text.x = element_text(angle = 45, hjust = 1))
  
  # Save
  out_file <- paste0("~/Documents/HCPDRBD/man_figs_pdf/deg_dot_plot/RBD_vs_HC_CellType_", cell_type, ".pdf")
  ggsave(filename = out_file, plot = p, device = "pdf", width = 10, height = 5)
}

Warning message:
“Scaling data with a low number of groups may produce misleading results”
Scale for colour is already present.
Adding another scale for colour, which will replace the existing scale.
Scale for size is already present.
Adding another scale for size, which will replace the existing scale.
Warning message:
“Scaling data with a low number of groups may produce misleading results”
Scale for colour is already present.
Adding another scale for colour, which will replace the existing scale.
Scale for size is already present.
Adding another scale for size, which will replace the existing scale.
Warning message:
“Scaling data with a low number of groups may produce misleading results”
Scale for colour is already present.
Adding another scale for colour, which will replace the existing scale.
Scale for size is already present.
Adding another scale for size, which will replace the existing scale.
Warning message:
“Scaling data with a low number of groups may produce misleading res

## GO Enrichment analysis on DEGs for each cell type

In [13]:
go_analysis <- function(deg_results_by_cluster, log2FC_threshold = 0.0){
    set.seed(384)
    # Define the database(s) to use for EnrichR, focusing on GO Biological Process
    dbs <- c("GO_Biological_Process_2021")
    
    # Initialize a list to store enrichment results
    enrichment_results <- list()
    marker_counts <- list() 
    
    # Loop over each cluster in deg_results_by_cluster to perform enrichment analysis
    for (cluster_name in names(deg_results_by_cluster)) {
      # Retrieve DEGs for the current cluster from deg_results_by_cluster
      cluster_data <- deg_results_by_cluster[[cluster_name]]
      
      # Select significant DEGs (e.g., adjusted p-value < 0.05 and log fold change threshold)
      significant_genes <- rownames(cluster_data[cluster_data$p_val_adj < 0.05 & cluster_data$avg_log2FC > log2FC_threshold, ])
      marker_counts[[cluster_name]] <- length(significant_genes)
        
      # Perform enrichment analysis if there are significant genes
      if (length(significant_genes) > 0) {
        enrich_result <- enrichr(significant_genes, dbs)
        go_results <- enrich_result[["GO_Biological_Process_2021"]]
        
        # Sort results by adjusted p-value and select the top 10 processes
        go_results <- go_results[order(go_results$Adjusted.P.value), ]
        top_processes <- head(go_results, 15)
    
        if (nrow(top_processes) != 0) {
            # Initialize list to store the average fold change for each GO term
            avg_fc_for_terms <- numeric(nrow(top_processes))
            
            # Calculate the average fold change for each GO term in top_processes
            for (j in 1:nrow(top_processes)) {
              if (!is.na(top_processes$Genes[j]) && top_processes$Genes[j] != "") {
                genes_in_term <- strsplit(as.character(top_processes$Genes[j]), ";")[[1]]  # Split the genes string by semicolon
                
                # Only keep genes that are present in cluster_data
                genes_in_term <- genes_in_term[genes_in_term %in% rownames(cluster_data)]
                  
                # Retrieve fold changes for these genes
                if (length(genes_in_term) > 0) {
                  fc_values <- cluster_data[genes_in_term, "avg_log2FC"]
                  avg_fc_for_terms[j] <- mean(fc_values, na.rm = TRUE)  # Calculate the average, ignoring NA values
                } else {
                  avg_fc_for_terms[j] <- NA  # If no genes are found, set to NA
                }
              } else {
                avg_fc_for_terms[j] <- NA  # If 'Genes' is NA, set to NA
              }
            }
        
            # Add the average fold changes as a new column in top_processes
            top_processes$Avg_Log2FC <- avg_fc_for_terms
        }
        # Store the top GO processes for this cluster
        enrichment_results[[cluster_name]] <- top_processes
      }
    }
    return(list(marker_counts = marker_counts, enrichment_results = enrichment_results))
}

In [14]:
set.seed(384)
deg_results_by_cluster_filt <- lapply(deg_results_by_cluster, function(df) {
  genes <- rownames(df)
  keep  <- (!grepl("^(RPL|RPS)", genes)) & (!grepl("^MT-", genes)) & (!grepl("\\.", genes))
  df[keep, , drop = FALSE]
})
output <- go_analysis(deg_results_by_cluster_filt, log2FC_threshold = 0.25)
list2env(output, envir = .GlobalEnv)

Uploading data to Enrichr... Done.
  Querying GO_Biological_Process_2021... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying GO_Biological_Process_2021... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying GO_Biological_Process_2021... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying GO_Biological_Process_2021... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying GO_Biological_Process_2021... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying GO_Biological_Process_2021... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying GO_Biological_Process_2021... Done.
Parsing results... Done.


<environment: R_GlobalEnv>

In [15]:
marker_counts

$`excitatory neurons`
[1] 3905

$`oligodendrocyte precursor cells`
[1] 359

$oligodendrocytes
[1] 4116

$`inhibitory neurons`
[1] 777

$astrocytes
[1] 4440

$microglia
[1] 1637

$`endothelia cells`
[1] 165

In [ ]:
set.seed(384) 
# Prepare matrices for plotting
cluster_names <- c("astrocytes", "endothelia cells", "excitatory neurons", "inhibitory neurons", "microglia", "oligodendrocytes", "oligodendrocyte precursor cells")
# Collect unique biological processes across all clusters
all_processes <- unique(unlist(lapply(enrichment_results, function(x) x$Term)))
enrichment_matrix <- matrix(
  NA, nrow = length(all_processes), ncol = length(cluster_names),
  dimnames = list(all_processes, cluster_names)
)
fc_matrix <- matrix(
  NA, nrow = length(all_processes), ncol = length(cluster_names),
  dimnames = list(all_processes, cluster_names)
)

# Fill the matrices with -log10(adjusted p-value) and Avg_Log2FC for each process and cluster
for (cluster_name in cluster_names) {
  if (!is.null(enrichment_results[[cluster_name]]) && nrow(enrichment_results[[cluster_name]]) > 0) {
    for (j in 1:nrow(enrichment_results[[cluster_name]])) {
      process <- enrichment_results[[cluster_name]]$Term[j]
      pval <- enrichment_results[[cluster_name]]$Adjusted.P.value[j]
      avg_log2fc <- enrichment_results[[cluster_name]]$Avg_Log2FC[j]
  
      # Fill p-value matrix with -log10(p-value)
      enrichment_matrix[process, cluster_name] <- -log10(pval)
      
      # Fill fold change matrix with Avg_Log2FC
      fc_matrix[process, cluster_name] <- avg_log2fc
      
    }
  }
}   

# Replace NA values with a low number (e.g., 0) for better plotting
enrichment_matrix[is.na(enrichment_matrix)] <- 0

output_pdf = "../../man_figs_pdf/allct_deg_go_RBDvsHC_logp+.pdf"
# Plot the heatmap using pheatmap
pheatmap::pheatmap(enrichment_matrix,
                   color = c(
                     colorRampPalette(c("white", '#ef3b2c'))(10),  # More resolution for small values
                     colorRampPalette(c('#ef3b2c','#cb181d','#a50f15','#67000d'))(90) # High contrast for larger values
                   ),
                   na_col = "grey",           # Use grey for NA values (e.g., processes not enriched in clusters)
                   main = "Enriched BP of Microglial State Signatures (-log10 p-value)",
                   cluster_rows = FALSE,       # Cluster processes for better visualization
                   cluster_cols = FALSE,      # Do not cluster clusters (retain the original order)
                   angle_col = 45,            # Rotate column names for readability
                   fontsize_row = 6,          # Font size for process names
                   fontsize_col = 8,
                   filename = output_pdf)         # Font size for cluster labels
print("Logp Plot saved")

# Replace NA values with a low number (e.g., 0) for better plotting
fc_matrix[is.na(fc_matrix)] <- 0

# Define the custom color palettes
# negative_colors <- colorRampPalette(c('#f7fbff','#deebf7','#c6dbef','#9ecae1','#6baed6','#4292c6', '#2171b5', '#084594'))(50)
negative_colors <- colorRampPalette(c('#084594','#2171b5','#4292c6','#6baed6','#9ecae1','#c6dbef', '#deebf7', '#f7fbff'))(50)
positive_colors <- colorRampPalette(c('#fff5f0','#fee0d2','#fcbba1','#fc9272','#fb6a4a','#ef3b2c','#cb181d','#99000d'))(50)

# Combine palettes with white at 0
custom_colors <- c(negative_colors, "white", positive_colors)

# Define custom breaks to center the color scale at 0
max_abs_value <- max(abs(fc_matrix))
breaks <- seq(-max_abs_value, max_abs_value, length.out = length(custom_colors) + 1)

output_pdf = "../../man_figs_pdf/allct_deg_go_RBDvsHC_log2fc+.pdf"
pheatmap::pheatmap(fc_matrix,
                   color = custom_colors,
                   breaks = breaks,              # Use custom breaks for diverging colors
                   na_col = "grey",              # Use grey for NA values
                   main = "Average Log2 Fold Change",
                   cluster_rows = FALSE,          # Cluster processes for better visualization
                   cluster_cols = FALSE,          # Cluster clusters for better visualization
                   angle_col = 45,               # Rotate column names for readability
                   fontsize_row = 6,             # Font size for process names
                   fontsize_col = 8,
                   filename = output_pdf)             # Font size for cluster labels
print("Log2FC File saved")                                                  

[1] "Logp Plot saved"
[1] "Log2FC File saved"


In [ ]:
library(openxlsx)
# Create a new workbook
wb <- createWorkbook()

for (cluster_name in cluster_names) {
    # Filter the data for each cluster
    cluster_data <- enrichment_results[[cluster_name]]
    
    # Add a worksheet with the cluster name
    addWorksheet(wb, sheetName = cluster_name)
    
    # Write the filtered data to the worksheet
    writeData(wb, sheet = cluster_name, cluster_data, rowNames = TRUE)
}

# Save the workbook to an Excel file
saveWorkbook(wb, file = "../../DEG_GO_Analysis/GO_RBD_vs_HC/GO_Results_RBD_vs_HC_allct+.xlsx", overwrite = TRUE)

In [ ]:
saveRDS(enrichment_results, "../../DEG_GO_Analysis/GO_RBD_vs_HC/cached_enrichment_results_allct+.rds")

# DEG analysis (PD vs RBD) for each cell type

In [13]:
set.seed(384)
Idents(ifnb_testct) <- "celltype"
deg_results_by_cluster <- list()


for (ct in unique(ifnb_testct$celltype)) {
  ct_subset <- subset(ifnb_testct, idents = ct)
  Idents(ct_subset) <- "disease"
  
  # Check if both "PD" and "HC" are present in the subset
  if (all(c("PD", "RBD") %in% unique(Idents(ct_subset)))) {
    # Perform differential expression analysis between "PD" and "HC" within this cluster
    deg_results <- FindMarkers(ct_subset, ident.1 = "PD", ident.2 = "RBD")
    deg_results_by_cluster[[ct]] <- deg_results
  } else {
    message(paste("Skipping ct", ct, "- does not contain both PD and RBD cells"))
  }
}

Skipping ct pericytes - does not contain both PD and RBD cells



In [ ]:
library(openxlsx)

# Create a new workbook
wb <- createWorkbook()

for (ct in unique(ifnb_testct$celltype)) {
    if (ct != "pericytes"){
        # Filter the data for each cluster
        cluster_data <- deg_results_by_cluster[[ct]] %>% filter(p_val_adj < 0.05) %>% arrange(desc(avg_log2FC))
        
        # Add a worksheet with the cluster name
        addWorksheet(wb, sheetName = ct)
        
        # Write the filtered data to the worksheet
        writeData(wb, sheet = ct, cluster_data, rowNames = TRUE)
    }
}

# Save the workbook to an Excel file
saveWorkbook(wb, file = "../../DEG_GO_Analysis/DEGs_PD_vs_RBD/Significant_DEGs_PD_vs_RBD_ct.xlsx", overwrite = TRUE)


### Dot plot for top DEGs per cell type

In [15]:
top10_tables <- list()

for (cell_type in names(deg_results_by_cluster)) {
  cat("\nTop 10/20 genes for cell type:", cell_type, "\n")
  
  # Get the dataframe
  df <- deg_results_by_cluster[[cell_type]] %>% filter(p_val_adj < 0.05)
    
  # Add gene names as a column if they're rownames
  df$gene <- rownames(df)
    
  df <- df[!grepl("^(LINC)", df$gene), ]
  df <- df[!grepl("^(NPIPA8|TTTY)", df$gene), ]
  df <- df[!grepl("^(MT-)", df$gene), ]
  df <- df[!grepl("\\.\\d+$", df$gene), ]
    
  if (cell_type %in% c("microglia", "inhibitory neurons", "astrocytes")){
      df <- df %>% filter(pct.1 >= 0.25 | pct.2 >= 0.25)
  }
  
  # Select top 10 genes with highest avg_log2FC
  top_genes <- df[order(-df$avg_log2FC), ][1:10, ]

  if (cell_type %in% c("inhibitory neurons", "excitatory neurons")){
     top_genes <- df[order(-df$avg_log2FC), ][1:20, ]
  }

  # Save to list
  top10_tables[[cell_type]] <- top_genes
  
  # Optionally print the top genes
  print(top_genes)
}


Top 10/20 genes for cell type: excitatory neurons 
                 p_val avg_log2FC pct.1 pct.2     p_val_adj     gene
XIST      0.000000e+00   5.507481 0.328 0.009  0.000000e+00     XIST
TSIX     3.984471e-156   4.571629 0.096 0.012 1.458356e-151     TSIX
C5orf17  2.667049e-169   3.554208 0.152 0.042 9.761668e-165  C5orf17
HIST1H1D  6.309214e-15   2.650105 0.010 0.001  2.309235e-10 HIST1H1D
ASB17     2.815175e-22   2.130123 0.019 0.005  1.030382e-17    ASB17
HLA-DMA   1.559506e-53   2.066855 0.077 0.029  5.707947e-49  HLA-DMA
MT1G      7.822864e-19   2.058192 0.023 0.008  2.863246e-14     MT1G
HIST1H4C 5.515767e-102   1.960780 0.115 0.036  2.018826e-97 HIST1H4C
HSPB1     2.331986e-91   1.804235 0.127 0.048  8.535301e-87    HSPB1
SERPINE1  5.491860e-09   1.623365 0.012 0.005  2.010076e-04 SERPINE1
HIST1H1C  8.500700e-07   1.557737 0.011 0.005  3.111341e-02 HIST1H1C
EIF3CL    5.622979e-10   1.535356 0.019 0.009  2.058066e-05   EIF3CL
ALLC      8.259291e-17   1.525172 0.025 0.010  3.02

In [16]:
custom_palette <- c('#2166ac','#92c5de','#f7f7f7','#f4a582','#b2182b')

# Loop through each cell type in the DEG results
for (cell_type in names(deg_results_by_cluster)) {
  top_genes <- top10_tables[[cell_type]][1:10,"gene"]
  if (cell_type %in% c("inhibitory neurons", "excitatory neurons")){
     top_genes <- top10_tables[[cell_type]][1:20,"gene"]
  }
  
  # Subset Seurat object for this cell type
  sub_obj <- subset(ifnb_testct, subset = celltype == cell_type)
  
  # Keep only genes that exist in this object
  available_genes <- intersect(top_genes, rownames(sub_obj))
  
  # Skip if no genes available
  if (length(available_genes) == 0) next

  # Get plot data first
  plot_obj <- DotPlot(
    object   = sub_obj,
    features = available_genes,
    group.by = "disease"
  )
  plot_data <- plot_obj$data
  
  # Calculate dynamic size breaks for percent expressed
  min_pct <- min(plot_data$pct.exp)
  max_pct <- max(plot_data$pct.exp)
  breaks <- seq(min_pct, max_pct, length.out = 5)
  labels <- paste0(round(breaks), "%")
  
  # Now plot with dynamic size scale
  p <- plot_obj +
    scale_color_gradientn(colors = custom_palette) +
    scale_size_area(
      max_size = 15,
      breaks = breaks,
      labels = labels,
      name = "Percent Expressed"
    ) +
    ggtitle(paste("Cell Type:", cell_type, "- Top 10 DEGs for PD vs RBD")) +
    RotatedAxis() +
    theme(axis.text.x = element_text(angle = 45, hjust = 1))
  
  # Save
  out_file <- paste0("~/Documents/HCPDRBD/man_figs_pdf/deg_dot_plot/PD_vs_RBD_CellType_", cell_type, ".pdf")
  ggsave(filename = out_file, plot = p, device = "pdf", width = 10, height = 5)
}

Warning message:
“Scaling data with a low number of groups may produce misleading results”
Scale for colour is already present.
Adding another scale for colour, which will replace the existing scale.
Scale for size is already present.
Adding another scale for size, which will replace the existing scale.
Warning message:
“Scaling data with a low number of groups may produce misleading results”
Scale for colour is already present.
Adding another scale for colour, which will replace the existing scale.
Scale for size is already present.
Adding another scale for size, which will replace the existing scale.
Warning message:
“Scaling data with a low number of groups may produce misleading results”
Scale for colour is already present.
Adding another scale for colour, which will replace the existing scale.
Scale for size is already present.
Adding another scale for size, which will replace the existing scale.
Warning message:
“Scaling data with a low number of groups may produce misleading res